# <center >Accidents corporels de la route
### Ouverture de la base de données et Structure des variables

Faire les importations nécessaires :

In [8]:
pip install graphviz


The following command must be run outside of the IPython shell:

    $ pip install graphviz

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [9]:
import pandas as pd
import numpy as np
import matplotlib as plt
import graphviz
import seaborn as sns

/Users/camille/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


ModuleNotFoundError: No module named 'graphviz'

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn import tree

/Users/camille/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/Users/camille/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_

Importer les 4 fichiers :

In [11]:
df_carac = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/e22ba475-45a3-46ac-a0f7-9ca9ed1e283a", sep = ";")
df_lieux = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/2ad65965-36a1-4452-9c08-61a6c874e3e6", sep = ";")
df_usagers = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/36b1b7b3-84b4-4901-9163-59ae8a9e3028", sep = ";")
df_vehicules = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/780cd335-5048-4bd6-a841-105b44eb2667", sep = ";")

Une première sélection a été réalisée. Nous avons éliminé les variables qui ne semblaient avoir aucun lien avec la gravité de l'accident.

In [12]:
df_carac = df_carac[["Num_Acc", "jour", "mois", "an", "hrmn", "lum", "dep", "com", "agg", "int", "atm", "col", "lat", "long"]]
df_lieux = df_lieux[["Num_Acc", "catr", "circ", "nbv", "surf", "prof", "plan", "infra", "situ", "vma"]] 
df_usagers = df_usagers[["Num_Acc", "id_vehicule", "place", "catu", "sexe", "grav", "an_nais", "trajet", "secu1"]]
df_vehicules = df_vehicules[["Num_Acc", "id_vehicule", "catv", "obs", "obsm", "choc", "occutc"]]

Fusionner les 4 bases de données :

In [13]:
df = df_carac.merge(df_lieux, how='left', on=['Num_Acc'])
df = df.merge(df_vehicules, how='inner', on=['Num_Acc'])
df = df.merge(df_usagers, how='left', on=['Num_Acc', 'id_vehicule'])
print(df.shape)

(135017, 36)


Il y a 135017 observations et 36 variables.

A l'aide du descriptif des modalités, nous remplaçons les modalités par leur nom complet pour une meilleure compréhension.

In [14]:
# Variables de la base Caracteristiques
df["lum"]= df['lum'].map({1:'Plein jour',
                        2:'Crépuscule ou aube',
                        3: "Nuit sans éclairage public",
                        4: "Nuit avec éclairage public non allumé",
                        5: "Nuit avec éclairage public allumé"})
df['agglo'] = df['agg'].map({1:'Hors agglomeration',
                             2:'Agglomeration'})
del df['agg']


df['int']=df['int'].map({1:'Hors intersection',
                        2:'Intersection en X',
                        3:'Intersection en T',
                        4:'Intersection en Y',
                        5:'Intersection à plus de 4 branches',
                        6:'Giratoire',
                        7:"Place",
                        8:'Passage à niveaux',
                        9:'Autre'})
df['atm'] = df['atm'].map({1:'Normale',
                             2:'Pluie legere',
                             3:'Pluie forte',
                             4:'Neige',
                          5:'Brouillard',
                          6:'Vent fort',
                          7:'Eblouissant',
                          8:'Couvert',
                          9:'Autre'})
df['collision'] = df['col'].map({1:'2 - frontale',
                             2:'2-arriere',
                             3:'2-cote',
                             4:'3-chaine',
                          5:'3-multiple',
                          6:'autre',
                          7:'Sans collision'})
del df['col']


# Variables de la base Usagers
df['catu'] = df['catu'].map({1:'Conducteur',
                             2:'Passager',
                             3:'Pieton'})
df['sexe'] = df['sexe'].map({1:'Homme',
                             2:'Femme'})
df['grav'] = df['grav'].map({1:'Indemne',
                             2:'Tue',
                             3:'Blesse hospitalise',
                             4:'Blesse leger'})


# Variables de la base Lieux
df['catr'] = df['catr'].map({1:'Autoroute',
                             2:'Route nationale',
                             3:'Route départementale',
                             4:'Voie Communale',
                          5:'Hors réseau public',
                          6:'Parc de stationnement',
                          7:'Routes de métropole urbaine',
                          9:'Autre'})
df["circ"]=df["circ"].map({1:"A sens unique",
                          2:"Bidirectionnelle",
                          3:'A chaussées séparées',
                          4:"Avec voies d'affectation variable"})
df["prof"]=df["prof"].map({1:"Plat",
                          2:"Pente",
                          3:'Sommet de côte',
                          4:"Bas de côte"})
df["plan"]=df["plan"].map({1:"Rectiligne",
                          2:"En courbe à gauche",
                          3:'En courbe à droite',
                          4:"En S"})
df['surf'] = df['surf'].map({1:'Normale',
                             2:'Mouillée',
                             3:'Flaques',
                             4:'Inondée',
                          5:'Enneigée',
                          6:'Boue',
                          7:'Verglecée',
                             8:'Corps gras - huile',
                          9:'Autre'})
df['situ'] = df['situ'].map({1:'Sur chaussée',
                             2:"Sur bande d'arrêt d'urgence",
                             3:'Sur accotement',
                             4:'Sur trottoir',
                          5:'Sur piste cyclable',
                          6:'Sur autre voie spéciale',
                          8:'Autre'})

# Création de nouvelles variables
df['age'] = 2019 - df['an_nais']

### Appropriation du jeu de données

In [15]:
df.columns

Index(['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'int',
       'atm', 'lat', 'long', 'catr', 'circ', 'nbv', 'surf', 'prof', 'plan',
       'infra', 'situ', 'vma', 'id_vehicule', 'catv', 'obs', 'obsm', 'choc',
       'occutc', 'place', 'catu', 'sexe', 'grav', 'an_nais', 'trajet', 'secu1',
       'agglo', 'collision', 'age'],
      dtype='object')

In [16]:
df.head(5)

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,int,atm,...,place,catu,sexe,grav,an_nais,trajet,secu1,agglo,collision,age
0,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,Hors intersection,Normale,...,2.0,Passager,Femme,Blesse leger,2002.0,0.0,1.0,Hors agglomeration,2-arriere,17.0
1,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,Hors intersection,Normale,...,1.0,Conducteur,Femme,Blesse leger,1993.0,5.0,1.0,Hors agglomeration,2-arriere,26.0
2,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,Hors intersection,Normale,...,1.0,Conducteur,Homme,Indemne,1959.0,0.0,1.0,Hors agglomeration,2-arriere,60.0
3,201900000002,30,11,2019,02:50,Nuit sans éclairage public,93,93066,Hors intersection,Normale,...,1.0,Conducteur,Femme,Blesse leger,1994.0,0.0,1.0,Hors agglomeration,autre,25.0
4,201900000003,28,11,2019,15:15,Plein jour,92,92036,Hors intersection,Normale,...,1.0,Conducteur,Homme,Indemne,1996.0,0.0,1.0,Hors agglomeration,3-chaine,23.0


In [17]:
df.dtypes

Num_Acc          int64
jour             int64
mois             int64
an               int64
hrmn            object
lum             object
dep             object
com             object
int             object
atm             object
lat             object
long            object
catr            object
circ            object
nbv              int64
surf            object
prof            object
plan            object
infra            int64
situ            object
vma              int64
id_vehicule     object
catv             int64
obs              int64
obsm             int64
choc             int64
occutc         float64
place          float64
catu            object
sexe            object
grav            object
an_nais        float64
trajet         float64
secu1          float64
agglo           object
collision       object
age            float64
dtype: object

In [18]:
#print(df.describe(include='all'))

### Etude des effectifs des modalités :

Ce jeu de données est complet. Les valeurs non renseignées sont indiquées par "-1".

In [19]:
colonne = ["lum", "agglo", "int", "collision", "atm", "catr", "circ", "surf", "prof", "plan", "infra", "situ",
           "catu", "sexe", "choc", "grav"]
def effectif_pourcentage():
    for elem in colonne :
        print("Pour la variable " + elem + " :")
        print((df[elem].value_counts())/len(df[elem])*100)
        print("\n")
#effectif_pourcentage()

**Regrouper les modalités ayant peu d'effectifs entre elles**

On regroupe les modalités entre elles lorsqu'elles forment moins de 5% de la population totale.

In [20]:
colonne = ["agglo", "sexe", "catr", "catu", "collision", "atm", "surf"]
def regroupement_modalites():
    for elem in colonne :
        frequence = df[elem].value_counts()/len(df[elem])*100
        for i in range(len(frequence)) :
            if frequence[i] < 5 :
                df.loc[df[elem]==frequence.index[i], elem] = "Autre"   
regroupement_modalites()
#effectif_pourcentage()

## Ajout de variables 

#### Variable gravite

On ajoute une variable qui donne une note de gravité à chaque accident nommée "gravite". 
- gravite = 0 si tout le monde ressort indemne de l'accident 
- gravite = 1 si il y a au moins un blessé léger 
- gravite = 2 si il y a au moins un blessé hospitalisé ou au moins un mort 

In [69]:
#CA CA MARCHE C'EST BON
def calcul_gravite(l_grav):
    # prend en entrée la liste des grav d'un même accident l_grav
    note = 0
    if (('Tue' in l_grav) or ('Blesse hospitalise' in l_grav)):
        #on check si on a un 'Tue' ou un 'Blesse hospitalise' (var grav) => note=2
        note = 2
        return (note) 
    elif ('Blesse leger' in l_grav):
        #si non on check si on a un 'Blesse leger' => note = 1
        note = 1
        return (note)
    # si non => note = 0
    return(note)

#TEST OK 
#calcul_gravite(['Indemne','Blesse hospitalise','Blesse leger','Tue'])
#calcul_gravite(['Indemne','Blesse leger','Indemne'])
#calcul_gravite(['Indemne','Indemne','Indemne'])
#calcul_gravite([])

1

In [87]:
#on veut récupérer toutes les données de gravité pour chaque personne d'un meme accident (var Num_Acc)
#on crée une variable gravite remplie de zero dans df 
df = df.assign(gravite=0)

#on récupère tous les Num_Acc distincts
accidents = df['Num_Acc'].unique()

for num in accidents: 
    #on boucle sur les Num_Acc pour récupérer toutes les valeurs de grav pour un meme accident 
    l_grav = list(df[df['Num_Acc'] == num]['grav'])
    note = calcul_gravite(l_grav) #on applique pour chaque accident la fonction calcul_gravite 
    for i in df[df['Num_Acc'] == num].index:
        #on insert la valeur dans chaque case de la nouvelle variable 
        df['gravite'][i] = note
    





/Users/camille/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [91]:
print(df)

             Num_Acc  jour  mois    an   hrmn  \
0       201900000001    30    11  2019  01:30   
1       201900000001    30    11  2019  01:30   
2       201900000001    30    11  2019  01:30   
3       201900000002    30    11  2019  02:50   
4       201900000003    28    11  2019  15:15   
...              ...   ...   ...   ...    ...   
135012  201900058837    27    11  2019  07:50   
135013  201900058838    30    11  2019  02:41   
135014  201900058839    30    11  2019  15:20   
135015  201900058840    29    11  2019  20:50   
135016  201900058840    29    11  2019  20:50   

                                          lum dep    com                int  \
0       Nuit avec éclairage public non allumé  93  93053  Hors intersection   
1       Nuit avec éclairage public non allumé  93  93053  Hors intersection   
2       Nuit avec éclairage public non allumé  93  93053  Hors intersection   
3                  Nuit sans éclairage public  93  93066  Hors intersection   
4               